In [5]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.image as mping
from string import Template
import math

pct = lambda x: '{:.2%}'.format(x)
dig = lambda x: '{:.2f}'.format(x)
exp = lambda x: math.exp(x)


def getLine(t,df):
    line = Template('$a ' '$b ' '$c ' '$d ')
    line = line.substitute(a=df.loc[t,'Security'], b=t, c=df.loc[t,'GICS Sub-Industry'], d=dig(df.loc[t,'Sharpe']))
    return line

pct = lambda x: '{:.2%}'.format(x)
dig = lambda x: '{:.2f}'.format(x)

#Get Symbol from Security
def getTicker(stock, df):
    p = df[df['Security'].str.contains(stock).tolist()]
    ticker = p.index.values[0]
    return ticker

#Get Security from Symbol
def getStock(ticker, df):
    stock = df.loc[ticker,'Security']
    stock = stock[0:30]
    return stock

now = datetime.now()
now = now.strftime('%F-%H')

# Read SP500 Tickers and Information
sp500_symbols = pd.read_csv ('data/S&P500-Symbols.csv')
sp500 = np.array(sp500_symbols['Symbol'])
info = pd.read_csv('data/S&P500-Info.csv', index_col=['Symbol'])
info = pd.DataFrame(info)


# Get Data from Yahoo Finance API (TO REVIEW)
#price = StockData(sp500, start_date, end_date)
#SPY  =  StockData('SPY', start_date, end_date)

custom_date_parser = lambda x: datetime.strptime(x,"%Y-%m-%d")
price = pd.read_csv('data/price.csv', index_col=['Date'], parse_dates=['Date'], date_parser=custom_date_parser)
price = pd.DataFrame(price)

tupper = pd.read_csv('data/tupper_2021-01-28.csv', index_col=['Unnamed: 0'])
tupper = pd.DataFrame(tupper)



tupper['Exp Return'] = tupper['Exp Return'].apply(exp).apply(pct)
tupper['Volatility'] = tupper['Volatility'].apply(exp).apply(pct)
tupper['Sharpe'] = tupper['Sharpe'].apply(dig)
tupper['Corr_value'] = tupper['Corr_value'].apply(dig)
tupper['mv_Returns'] = tupper['mv_Returns'].apply(exp).apply(pct)
tupper['mv_Volatility'] = tupper['mv_Volatility'].apply(exp).apply(pct)
tupper['or_Returns'] = tupper['or_Returns'].apply(exp).apply(pct)
tupper['or_Volatility'] = tupper['or_Volatility'].apply(exp).apply(pct)



print('-'*80)
print('-'*80)

print('-'*80)
print("Data calculated on "+now)
print('-'*80)
print('\n')

stock = input('Enter your stock: ')
ticker = getTicker(stock, info)
stock = getStock(ticker,info)

#Reading pictures
img = mping.imread('data/img/'+ticker+'_optimal_porfolio.png')

print('-'*80)
print('\n')


r = tupper.loc[ticker,'Exp Return']
v = tupper.loc[ticker,'Volatility']
s = tupper.loc[ticker,'Sharpe']
b = tupper.loc[ticker,'Min_Corr']
c = tupper.loc[ticker,'Corr_value']
i = tupper.loc[ticker,'GICS Sub-Industry']
p0 = tupper.loc[ticker,'a0']
p1 = tupper.loc[ticker,'a1']
p2 = tupper.loc[ticker,'a2']
p3 = tupper.loc[ticker,'a3']
p4 = tupper.loc[ticker,'a4']
p5 = tupper.loc[ticker,'a5']
p6 = tupper.loc[ticker,'a6']
p7 = tupper.loc[ticker,'a7']
p8 = tupper.loc[ticker,'a8']
p9 = tupper.loc[ticker,'a9']
mvr = tupper.loc[ticker,'mv_Returns']
mvv = tupper.loc[ticker,'mv_Volatility']
mw0= dig(tupper.loc[ticker,'mv_w0'])
mw1= dig(tupper.loc[ticker,'mv_w1'])
mw2= dig(tupper.loc[ticker,'mv_w2'])
mw3= dig(tupper.loc[ticker,'mv_w3'])
mw4= dig(tupper.loc[ticker,'mv_w4'])
mw5= dig(tupper.loc[ticker,'mv_w5'])
mw6= dig(tupper.loc[ticker,'mv_w6'])
mw7= dig(tupper.loc[ticker,'mv_w7'])
mw8= dig(tupper.loc[ticker,'mv_w8'])
mw9= dig(tupper.loc[ticker,'mv_w9'])
opr = tupper.loc[ticker,'or_Returns']
opv = tupper.loc[ticker,'or_Volatility']
ow0= tupper.loc[ticker,'or_w0']
ow1= tupper.loc[ticker,'or_w1']
ow2= tupper.loc[ticker,'or_w2']
ow3= tupper.loc[ticker,'or_w3']
ow4= tupper.loc[ticker,'or_w4']
ow5= tupper.loc[ticker,'or_w5']
ow6= tupper.loc[ticker,'or_w6']
ow7= tupper.loc[ticker,'or_w7']
ow8= tupper.loc[ticker,'or_w8']
ow9= tupper.loc[ticker,'or_w9']       

t = tupper[tupper['GICS Sub-Industry']== i].sort_values(by='Sharpe', ascending = False).head(10)
t = t.loc[:,['Security','Sharpe','GICS Sub-Industry']]

c = tupper.loc[ticker,['a0','a1','a2','a3','a4','a5','a6','a7','a8','a9']]
p = tupper.loc[ticker,['mv_w0','mv_w1','mv_w2','mv_w3','mv_w4','mv_w5','mv_w6','mv_w7','mv_w8','mv_w9']]
x = tupper.loc[ticker,['or_w0','or_w1','or_w2','or_w3','or_w4','or_w5','or_w6','or_w7','or_w8','or_w9']]


new_df = pd.DataFrame(0, columns = c, index = ['Minimum Volatility Porfolio', 'Maximum Sharpe Ration'])
new_df.iloc[0,:] = np.array(p)
new_df.iloc[1,:] = np.array(x)
new_df.iloc[0,:] = new_df.iloc[0,:].apply(dig)
new_df.iloc[1,:] = new_df.iloc[1,:].apply(dig)


print('-'*80)
print('Modern Portfolio Theory(MPT)')
print('Based in the '+stock+' stock annual evolution, those are the 9 stock with the highest Sharpe ratio and minimum correlation:')
print('\n')
print(tupper.loc[[p0,p1,p2,p3,p4,p5,p6,p7,p8,p9],['Security','Exp Return','Volatility','Sharpe','GICS Sub-Industry']])
print('\n')
print('-'*80)
print('Ranking of Sharpe ration in the same industry:')
print('\n')
print(t)
print('-'*80)
print('Porfolio with MAXIMUM Sharpe ratio:')
print('\n')
print('Annual Expected Return: '+mvr)
print('Annual Expected Volatility: '+mvv)
print('\n')
print('Porfolio with MINIMUM Volatility:')
print('\n')
print('Annual Expected Return: '+opr)
print('Annual Expected Volatility: '+opv)
print('\n')
print(new_df)
plt.subplots(figsize=(10, 10))
plt.imshow(img) 
print('\n')
print('-'*80)
print('FAQ')
print('1 - Sharpe ratio: ratio is used to help investors understand the return of an investment compared to its risk.\nThe ratio is the average return earned in excess of the risk-free rate per unit of volatility or total risk.\nVolatility is a measure of the price fluctuations of an asset or portfolio.' )
print('\n')
print('2 - Correlation, in the finance and investment industries, is a statistic that measures the degree to which \ntwo securities move in relation to each other. Correlations are used in advanced portfolio management, computed \nas the correlation coefficient, which has a value that must fall between -1.0 and +1.0.')
print('\n')
print("3 - Modern Portfolio Theory(MPT) or also known as mean-variance analysis is a mathematical process \nwhich allows the user to maximize returns for a given risk level.") 
print("It was formulated by H. Markowitz and while it is not the only optimization technique known, \nit is the most widely used.")
print("\nMPT encourages diversification of assets. It says that a high variance asset A if combined \nwith diverse assets B and C, where A, B and C have little to no correlation, can give us \na portfolio with low variance on returns.")


--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Data calculated on 2021-01-29-17
--------------------------------------------------------------------------------


Enter your stock: CSCO


IndexError: index 0 is out of bounds for axis 0 with size 0

In [73]:
tupper

,Exp Return,Volatility,Sharpe,Min_Corr,Corr_value,Security,GICS Sector,GICS Sub-Industry,a0,a1,...,or_w0,or_w1,or_w2,or_w3,or_w4,or_w5,or_w6,or_w7,or_w8,or_w9
MMM,100.52%,143.71%,0.01,NEM,0.16,3M Company,Industrials,Industrial Conglomerates,MMM,NLOK,...,0.02,0.18,0.08,0.02,0.06,0.14,0.20,0.00,0.00,0.30
ABT,128.39%,145.88%,0.66,VNT,0.13,Abbott Laboratories,Health Care,Health Care Equipment,ABT,VNT,...,0.01,0.31,0.01,0.01,0.07,0.12,0.06,0.10,0.23,0.07
ABBV,128.90%,140.30%,0.75,NEM,0.10,AbbVie Inc.,Health Care,Pharmaceuticals,ABBV,REGN,...,0.09,0.12,0.12,0.23,0.07,0.20,0.03,0.00,0.04,0.09
ABMD,177.39%,156.19%,1.29,VNT,0.01,ABIOMED Inc,Health Care,Health Care Equipment,ABMD,MU,...,0.21,0.05,0.37,0.06,0.13,0.05,0.01,0.03,0.04,0.05
ACN,123.70%,149.10%,0.53,VNT,0.20,Accenture plc,Information Technology,IT Consulting & Other Services,ACN,REGN,...,0.01,0.03,0.14,0.21,0.16,0.21,0.13,0.00,0.02,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,107.04%,151.63%,0.16,KR,-0.01,Yum! Brands Inc,Consumer Discretionary,Restaurants,YUM,CLX,...,0.00,0.07,0.14,0.10,0.08,0.20,0.01,0.03,0.16,0.21
ZBRA,153.96%,161.85%,0.90,NEM,0.16,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,ZBRA,REGN,...,0.05,0.01,0.15,0.08,0.06,0.18,0.17,0.01,0.14,0.15
ZBH,107.09%,161.43%,0.14,CPB,-0.07,Zimmer Biomet,Health Care,Health Care Equipment,ZBH,NLOK,...,0.01,0.14,0.14,0.11,0.11,0.16,0.10,0.01,0.00,0.21
ZION,99.66%,178.36%,-0.01,ATVI,-0.10,Zions Bancorp,Financials,Regional Banks,ZION,PGR,...,0.02,0.00,0.31,0.05,0.00,0.23,0.12,0.16,0.01,0.10


In [69]:
ow0+ow1+ow2+ow3+ow4+ow5+ow6+ow7+ow8+ow9

1.0000000000000002

In [43]:
r

'2.73%'

<bound method Index._add_logical_methods.<locals>._make_logical_function.<locals>.logical_func of Index(['Exp Return', 'Volatility', 'Sharpe', 'Min_Corr', 'Corr_value',
       'Security', 'GICS Sector', 'GICS Sub-Industry', 'a0', 'a1', 'a2', 'a3',
       'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'mv_Returns', 'mv_Volatility',
       'mv_w0', 'mv_w1', 'mv_w2', 'mv_w3', 'mv_w4', 'mv_w5', 'mv_w6', 'mv_w7',
       'mv_w8', 'mv_w9', 'or_Returns', 'or_Volatility', 'or_w0', 'or_w1',
       'or_w2', 'or_w3', 'or_w4', 'or_w5', 'or_w6', 'or_w7', 'or_w8', 'or_w9'],
      dtype='object')>

or_w0      0.300416
or_w1     0.0310473
or_w2    0.00963759
or_w3      0.189774
or_w4      0.200498
or_w5     0.0183374
or_w6     0.0824227
or_w7     0.0222656
or_w8      0.139754
or_w9    0.00584753
Name: AAPL, dtype: object

,Security,Sharpe,GICS Sub-Industry
TSLA,"Tesla, Inc.",0.44,Automobile Manufacturers
ETSY,Etsy,0.22,Internet & Direct Marketing Retail
AMZN,Amazon.com Inc.,0.07,Internet & Direct Marketing Retail
LB,L Brands Inc.,0.06,Apparel Retail
POOL,Pool Corporation,0.05,Distributors
TSCO,Tractor Supply Company,0.05,Specialty Stores
CMG,Chipotle Mexican Grill,0.05,Restaurants
TGT,Target Corp.,0.04,General Merchandise Stores
EBAY,eBay Inc.,0.04,Internet & Direct Marketing Retail
DG,Dollar General,0.04,General Merchandise Stores
